In [ ]:
import json
import pandas as pd

### 2018

In [ ]:
with open('FIFA/data-2018.json') as file:
    data = json.load(file)
team_map = {team['id']: team['name'] for team in data['teams']}
stadium_map = {stadium['id']: stadium['city'] for stadium in data['stadiums']}
matches_list = []

def process_matches(match_data, edition_name, round_name):
    for match in match_data:
        match_info = {
            'id_match': match['name'],
            'home_team': team_map[match['home_team']],
            'away_team': team_map[match['away_team']],
            'home_result': match['home_result'],
            'away_result': match['away_result'],
            'date': match['date'],
            'round': round_name,
            'city': stadium_map[match['stadium']],
            'édition': edition_name
        }
        matches_list.append(match_info)

for group_key, group_data in data['groups'].items():
    process_matches(group_data['matches'], '2018-RUSSIA', 'Group')

knockout_rounds = {
    'round_16': 'Round of 16',
    'round_8': 'Quarter-finals',
    'round_4': 'Semi-finals',
    'round_2_loser': 'Third place play-off',
    'round_2': 'Final'
}

for round_key, round_name in knockout_rounds.items():
    process_matches(data['knockout'][round_key]['matches'], '2018-RUSSIA', round_name)

df_2018 = pd.DataFrame(matches_list)

print(df_2018.head())

### 1930-2010

In [ ]:
file_path = 'FIFA/matches-19302010.csv'
data = pd.read_csv(file_path)

data['id_match'] = range(1, len(data) + 1)

def extract_results(score):
    try:
        score = score.split(' ')[0]  
        home_score, away_score = map(int, score.split('-'))
        return home_score, away_score
    except (ValueError, IndexError):
        return None, None  

data[['home_result', 'away_result']] = data['score'].apply(lambda x: pd.Series(extract_results(x)))
data['date'] = pd.to_datetime(data['year'].astype(str) + '-01-01')
data['city'] = data['venue'].str.split('.').str[0]
df_19302010 = data[['id_match', 'team1', 'team2', 'home_result', 'away_result', 'date', 'round', 'city', 'edition']]
df_19302010.rename(columns={
    'id_match': 'id_match',
    'team1': 'home_team',          
    'team2': 'away_team',          
    'edition': 'édition'}, inplace=True)
df_19302010 = df_19302010[df_19302010['édition'] != '2014-BRAZIL']
print(df_19302010.head())

### 2022

In [ ]:
file_path = 'FIFA/fifa2022.csv'
df = pd.read_csv(file_path)

df_2022 = pd.DataFrame()

df_2022['id_match'] = range(1, len(df) + 1)

df_2022['home_team'] = df['team1']
df_2022['away_team'] = df['team2']
df_2022['home_result'] = df['number of goals team1']
df_2022['away_result'] = df['number of goals team2']
df_2022['date'] = df['date']
df_2022['round'] = df['category']
    df_2022['city'] = 'Inconnue' 
df_2022['édition'] = '2022-QATAR' 
print(df_2022.head())

### 2014

In [ ]:
import pandas as pd

df = pd.read_csv('FIFA/worldcupmatches2014.txt', sep=';', encoding='utf-8')

df_2014 = pd.DataFrame({
    'id_match': range(1, len(df) + 1),  
    'home_team': df['Home Team Name'],
    'away_team': df['Away Team Name'],
    'home_result': df['Home Team Goals'],
    'away_result': df['Away Team Goals'],
    'date': df['Datetime'],  
    'round': df['Stage'],  
    'city': df['City'],
    'édition': '2014-BRAZIL' 
})
print(df_2014.head())

### Complete

In [ ]:
combined_df = pd.concat([df_19302010, df_2014, df_2018, df_2022], ignore_index=True)
combined_df.reset_index(drop=True, inplace=True)
combined_df['id_match'] = range(1, len(combined_df) + 1)
combined_df

## BONUS D'UNE VIE

- quel pays a gagné le + de matchs en coupe du monde ?
- quel est le top 5 des vainqueurs de coupe du monde (finale) ?
- quel est le plus grand écart de buts dans un match ?
- combien de matchs nuls au total ?
- quel pays est allé le plus souvent en phase finale (à partir des 8ème de finale) de cdm ?